In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import tqdm as notebook_tqdm
%matplotlib inline

# Don't Show Warning Messages
import warnings
warnings.filterwarnings('ignore')

In [9]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on:
    """

    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def check_type_forward(self, in_types):
        assert len(in_types) == 3

        x0_type, x1_type, y_type = in_types
        assert x0_type.size() == x1_type.shape
        assert x1_type.size()[0] == y_type.shape[0]
        assert x1_type.size()[0] > 0
        assert x0_type.dim() == 2
        assert x1_type.dim() == 2
        assert y_type.dim() == 1

    def forward(self, x0, x1, y):
        self.check_type_forward((x0, x1, y))

        # euclidian distance
        diff = x0 - x1
        dist_sq = torch.sum(torch.pow(diff, 2), 1)
        dist = torch.sqrt(dist_sq)

        mdist = self.margin - dist
        dist = torch.clamp(mdist, min=0.0)
        loss = y * dist_sq + (1 - y) * torch.pow(dist, 2)
        loss = torch.sum(loss) / 2.0 / x0.size()[0]
        return loss

In [11]:
import argparse
import torch
import torchvision.datasets as dsets
import random
import numpy as np
import time
import matplotlib.pyplot as plt

from siarec.model import SiameseNetwork
from torch.autograd import Variable
from torchvision import transforms


class Dataset(object):

    def __init__(self, x0, x1, label):
        self.size = label.shape[0]
        self.x0 = torch.from_numpy(x0)
        self.x1 = torch.from_numpy(x1)
        self.label = torch.from_numpy(label)

    def __getitem__(self, index):
        return (self.x0[index],
                self.x1[index],
                self.label[index])

    def __len__(self):
        return self.size





In [20]:
## Config
epoch = 1
batchsize = 8
data_path = './data/'

In [4]:
import tqdm as notebook_tqdm
from siarec.dataset import ASRDataset
from siarec.model import SiameseNetwork
import os

In [17]:
# iterate over files in directory
# train_data = []
# test_data = []

# for files in os.listdir(data_path+"train"):
#     speaker = files.split("_")[0][-4:]
#     train_data.append((data_path+"train/"+files, int(speaker)%45))

# for files in os.listdir(data_path+"test"):
#     speaker = files.split("_")[0][-4:]
#     train_data.append((data_path+"test/"+files, int(speaker)%45))

# train_df = pd.DataFrame(train_data, columns=["path", "speaker"])
# test_df = pd.DataFrame(test_data, columns=["path", "speaker"])

# train_df.to_csv("./data/train/train.csv", index=False)
# test_df.to_csv("./data/test/test.csv", index=False)
